In [2]:
! pip install datasets

  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached pyarrow-18.1.0-cp312-cp312-win_amd64.whl.metadata (3.4 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
Using cached datasets-3.2.0-py3-none-any.whl (480 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
   ---------------------------------------- 0.0/179.3 kB ? eta -:--:--
   -- ------------------------------------- 10.2/179.3 kB ? eta -:--:--
   -------- ------------------------------ 41.0/179.3 kB 393.8 kB/s eta 0:00:01
   --------------- ----------------------- 71.7/179.3 kB 563.7 kB/s eta 0:00:01
   ------------------------------------ - 174.1/179.3 kB 952.6 kB/s eta 0:00:01
   -------------------------------------- 179.3/179.3 kB 899.0 kB/s eta 0:00:00
Using cached multiprocess-0.70.16-py312-none-any.whl (146 kB)
Using cached pyarrow-18.1.0-c

In [28]:
from reuters21578 import Reuters21578
import torch

! pip install transformers
from transformers import BertTokenizer, BertModel


In [24]:
from reuters21578 import Reuters21578

# Create an instance of the dataset with a specific configuration
dataset = Reuters21578(config_name='ModApte')

# Download and prepare the dataset
dataset.download_and_prepare()

# Load the dataset
datasets = dataset.as_dataset()

# Access the train split
train_dataset = datasets['train']

# Print the labels for the first few examples
topics_count_dict = {}
for example in train_dataset:
    for topic in example['topics']:
        if topic not in topics_count_dict:
            topics_count_dict[topic] = 0
        topics_count_dict[topic] += 1

In [69]:
for i in range(10):
    print(train_dataset[i]['topics'])

['cocoa']
['grain', 'wheat', 'corn', 'barley', 'oat', 'sorghum']
['veg-oil', 'linseed', 'lin-oil', 'soy-oil', 'sun-oil', 'soybean', 'oilseed', 'corn', 'sunseed', 'grain', 'sorghum', 'wheat']
[]
['earn']
['acq']
['earn']
['earn', 'acq']
['earn']
['earn']


In [68]:
# print top 10 topics

sorted_topics = sorted(topics_count_dict.items(), key=lambda x: x[1], reverse=True)
for i in range(100):
    print(sorted_topics[i])

('earn', 2877)
('acq', 1650)
('money-fx', 538)
('grain', 433)
('crude', 389)
('trade', 369)
('interest', 347)
('wheat', 212)
('ship', 197)
('corn', 182)
('money-supply', 140)
('dlr', 131)
('sugar', 126)
('oilseed', 124)
('coffee', 111)
('gnp', 101)
('gold', 94)
('veg-oil', 87)
('soybean', 78)
('livestock', 75)
('nat-gas', 75)
('bop', 75)
('cpi', 69)
('cocoa', 55)
('reserves', 55)
('carcass', 50)
('copper', 47)
('jobs', 46)
('yen', 45)
('ipi', 41)
('iron-steel', 40)
('cotton', 39)
('barley', 37)
('rubber', 37)
('gas', 37)
('rice', 35)
('alum', 35)
('meal-feed', 30)
('palm-oil', 30)
('sorghum', 24)
('retail', 23)
('silver', 21)
('zinc', 21)
('pet-chem', 20)
('wpi', 19)
('tin', 18)
('rapeseed', 18)
('stg', 17)
('housing', 16)
('strategic-metal', 16)
('hog', 16)
('orange', 16)
('lead', 15)
('soy-oil', 14)
('heat', 14)
('soy-meal', 13)
('fuel', 13)
('lei', 12)
('sunseed', 11)
('lumber', 10)
('dmk', 10)
('tea', 9)
('income', 9)
('oat', 8)
('nickel', 8)
('l-cattle', 6)
('sun-oil', 5)
('platin

In [78]:


# Load pre-trained model tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def split_doc_by_window(doc, window_size):
    # Initialize the BERT tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Tokenize the document
    tokens = tokenizer.tokenize(doc)
    chunks = []
    chunk = []
    current_chunk_size = 0

    for token in tokens:
        chunk.append(token)
        current_chunk_size += 1

        if token == '.' and current_chunk_size >= window_size:
            chunks.append(' '.join(chunk))
            chunk = []
            current_chunk_size = 0

    if len(chunk) > 0:
        chunks.append(' '.join(chunk))

    return chunks


def encode_chunks(chunks):
    encoding = tokenizer.batch_encode_plus( chunks,# List of input texts
    padding=True,              # Pad to the maximum sequence length
    truncation=True,           # Truncate to the maximum sequence length if necessary
    return_tensors='pt',      # Return PyTorch tensors
    add_special_tokens=True    # Add special tokens CLS and SEP
    )

    input_ids = encoding['input_ids']  # Tokenized and encoded sentences
    attention_mask = encoding['attention_mask']  # Attention mask

    # Generate embeddings using BERT model

    model = BertModel.from_pretrained('bert-base-uncased')

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        chunks_embedding = outputs.last_hidden_state.mean(dim=1)

    return chunks_embedding

def split_and_encode_doc(doc, window_size):
    chunks = split_doc_by_window(doc, window_size)
    chunks_embedding = encode_chunks(chunks)

    return chunks_embedding

def get_cos_sim_mat(embedding_1, embedding_2):
    cos_sim_matrix = embedding_1 @ embedding_2.T

    norm_1 = torch.norm(embedding_1, dim=1)
    norm_2 = torch.norm(embedding_2, dim=1)

    norm_mult_div = norm_1.reshape(-1, 1) @ norm_2.reshape(-1, 1).T

    cos_sim_matrix = cos_sim_matrix / norm_mult_div

    return cos_sim_matrix


windows = [10, 30, 50, 100, float('inf')]
doc_1 = train_dataset[1]['text']
doc_2 = train_dataset[2]['text']
for window in windows:
    chunks_embedding_1 = split_and_encode_doc(doc_1, window)
    chunks_embedding_2 = split_and_encode_doc(doc_2, window)

    cos_sim_matrix = get_cos_sim_mat(chunks_embedding_1, chunks_embedding_2)
    print(f'Window size: {window}, Cosine similarity mean: {cos_sim_matrix.mean()}, Cosine similarity max: {cos_sim_matrix.max()}')

Window size: 10, Cosine similarity mean: 0.7154720425605774, Cosine similarity max: 0.9022217988967896
Window size: 30, Cosine similarity mean: 0.7808124423027039, Cosine similarity max: 0.8669080138206482
Window size: 50, Cosine similarity mean: 0.7851669192314148, Cosine similarity max: 0.878635585308075
Window size: 100, Cosine similarity mean: 0.8139405846595764, Cosine similarity max: 0.9040043354034424
Window size: inf, Cosine similarity mean: 0.9119062423706055, Cosine similarity max: 0.9119062423706055


In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

class Embedder:
    def __init__(self):
        pass

    def embed(self, chunks):
        pass

class BertEmbedder(Embedder):
    def __init__(self):
        self.model = BertModel.from_pretrained('bert-base-uncased')
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def embed(self, chunks):
        encodings = self.tokenizer.batch_encode_plus(chunks, padding=True, truncation=True, return_tensors='pt', add_special_tokens=True)
        input_ids = encodings['input_ids']
        attention_mask = encodings['attention_mask']

        with torch.no_grad():
            outputs = self.model(input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state.mean(dim=1)

        return embeddings
    
class TFIDFEmbedder(Embedder):
    def __init__(self):
        self.vectorizer = TfidfVectorizer()

    def embed(self, chunks):
        embeddings = self.vectorizer.fit_transform(chunks)
        return embeddings
    
    

class CustomClusteringModel:
    def __init__(self, window_size):
        self.window_size = window_size

    def data(self, dataset):
        self.dataset = dataset
    
    def fit(self, embbeders, weights = 'inv_sqrt_len'):
        if weights == 'inv_sqrt_len':
            weights_func = self._inv_sqrt_len_weights

        for doc in self.dataset:
            chunks = self._split_doc_by_window(doc['text'], self.window_size)
            embedding = []
            for embedder in embbeders:
                embed = embedder.embed(chunks)
                weights = weights_func(embed)
                embedding.append(embed * weights)

            self.embeddings.append(embedding)

    def cluster(self, n, method='kmeans'):
        if method == 'kmeans':
            self.kmeans = KMeans(n_clusters=n, random_state=0).fit(self.embeddings)
            self.labels = self.kmeans.labels_
        else:
            raise ValueError(f'Clustering method {method} is not supported')





    def _split_doc_by_window(doc, window_size):
        # Initialize the BERT tokenizer
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        # Tokenize the document
        tokens = tokenizer.tokenize(doc)
        chunks = []
        chunk = []
        current_chunk_size = 0

        for token in tokens:
            chunk.append(token)
            current_chunk_size += 1

            if token == '.' and current_chunk_size >= window_size:
                chunks.append(' '.join(chunk))
                chunk = []
                current_chunk_size = 0

        if len(chunk) > 0:
            chunks.append(' '.join(chunk))

        return chunks

    def _inv_sqrt_len_weights(self, embeddings):
        return 1/(len(embeddings[0])**0.5)